In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.externals import joblib
from IPython.display import display, Image

import pandas as pd
import os

import numpy as np
import face_recognition
import math
import time
import cv2

import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
weight_model = 'weight_predictor_landmark.model'
height_model = 'height_predictor_landmark.model'
bmi_model = 'bmi_predictor_landmark.model'

In [4]:
height_model = joblib.load(height_model)
weight_model = joblib.load(weight_model)
bmi_model = joblib.load(bmi_model)

In [5]:
label_file = "./data.csv"

In [6]:
import pandas as pd
import os

In [7]:
img_paths = []
for img_id in range(3700, 4206):
    img_paths.append(['img_%s.bmp' % img_id , '/home/harry/Downloads/TA/release/Data/Images/img_%s.bmp' % img_id])

In [8]:
for i in img_paths:
    print(i)

['img_3700.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3700.bmp']
['img_3701.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3701.bmp']
['img_3702.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3702.bmp']
['img_3703.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3703.bmp']
['img_3704.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3704.bmp']
['img_3705.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3705.bmp']
['img_3706.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3706.bmp']
['img_3707.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3707.bmp']
['img_3708.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3708.bmp']
['img_3709.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3709.bmp']
['img_3710.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3710.bmp']
['img_3711.bmp', '/home/harry/Downloads/TA/release/Data/Images/img_3711.bmp']
['img_3712.bmp', '/home/harry/Downloads/TA/release/Data/Images/i

In [9]:
len(img_paths)

506

In [10]:
image_df = pd.DataFrame(img_paths,columns=['name','path'])

In [11]:
image_df

,name,path
0,img_3700.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
1,img_3701.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
2,img_3702.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
3,img_3703.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
4,img_3704.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
...,...,...
501,img_4201.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
502,img_4202.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
503,img_4203.bmp,/home/harry/Downloads/TA/release/Data/Images/i...
504,img_4204.bmp,/home/harry/Downloads/TA/release/Data/Images/i...


In [12]:
profile_df = pd.read_csv(label_file)

In [13]:
profile_df.name

0          img_0.bmp
1          img_1.bmp
2          img_2.bmp
3          img_3.bmp
4          img_4.bmp
            ...     
4201    img_4201.bmp
4202    img_4202.bmp
4203    img_4203.bmp
4204    img_4204.bmp
4205    img_4205.bmp
Name: name, Length: 4206, dtype: object

In [14]:
len(profile_df)

4206

## How is BMI calculated?
```
BMI = (Weight in Kg) / ((Height in Meters) ^ 2)
```


All the training images are kept in the following directory

In [15]:
data_folder = "Images"

In [16]:
from glob import glob
all_files = glob(data_folder+"/*")

all_jpgs = sorted([img for img in all_files if ".jpg" in img or ".jpeg" in img or "bmp" in img])

print("Total {} photos ".format(len(all_jpgs)))

Total 0 photos 


In [17]:
from pathlib import Path as p

In [18]:
def get_index_of_digit(string):
    import re
    match = re.search("\d", p(string).stem)
    return match.start(0)

In [19]:
id_path = [(p(images).stem[:(get_index_of_digit(p(images).stem))],images) for  images in all_jpgs ]

In [20]:
data_df = image_df.merge(profile_df) ## merged the training images with their profile

In [21]:
data_df.drop(['Unnamed: 0'], axis=1)

,name,path,bmi,gender,weight,height
0,img_3700.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,48.277006,Male,144,1.73
1,img_3701.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,29.834105,Male,85,1.69
2,img_3702.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,36.019158,Female,97,1.64
3,img_3703.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,28.032301,Female,70,1.58
4,img_3704.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,48.779592,Male,158,1.80
...,...,...,...,...,...,...
501,img_4201.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,34.078947,Male,115,1.84
502,img_4202.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,34.564776,Female,94,1.65
503,img_4203.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,27.432362,Female,69,1.59
504,img_4204.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,40.492800,Male,125,1.76


## Extract face embedding using facenet pretrained architecture

In [22]:
import face_recognition
import numpy as np

In [23]:
def get_face_encoding(image_path):
    print(image_path)
#     img = imageio.imread('test-face.png')
    img = cv2.imread(image_path)
    landmarks = extract_face_landmarks(img)
    if landmarks is None:
        print("no face found !!!")
        
        return np.zeros(2278).tolist()
    print(landmarks.shape)
    min = 10000009
    max = -1
    distances = []
    for i in range(68):
        for j in range(68):
            if i<j:
                distances.append(math.sqrt((landmarks[i][0] - landmarks[j][0]) ** 2 + (landmarks[i][1] - landmarks[j][1]) ** 2))
    distances = np.array(distances)
    print(distances.shape)
    return distances

In [24]:
all_faces = []
all_height = []
all_weight = []
all_bmi = []

In [25]:
## all labels
y_height = data_df.height.values
y_weight = data_df.weight.values
y_BMI = data_df.bmi.values

In [26]:
def predict_height_width_BMI(test_image,height_model,weight_model,bmi_model):
    test_array = np.expand_dims(np.array(get_face_encoding(test_image)),axis=0)
    height = np.asscalar(np.exp(height_model.predict(test_array)))
    all_height.append(height)
    weight = np.asscalar(np.exp(weight_model.predict(test_array)))
    all_weight.append(weight)
    bmi = np.asscalar(np.exp(bmi_model.predict(test_array)))
    all_bmi.append(bmi)
    return {'height':height,'weight':weight,'bmi':bmi}
    

In [27]:
for images in data_df.path:
    predict_height_width_BMI(images,height_model,weight_model,bmi_model)

/home/harry/Downloads/TA/release/Data/Images/img_3700.bmp
no face found !!!
/home/harry/Downloads/TA/release/Data/Images/img_3701.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3702.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3703.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3704.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3705.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3706.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3707.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3708.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3709.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3710.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3711.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3712.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Dat

(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3811.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3812.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3813.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3814.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3815.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3816.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3817.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3818.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3819.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3820.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3821.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3822.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3823.bmp
(68, 2)
(2278,)
/home/harry/Downloads/

(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3922.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3923.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3924.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3925.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3926.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3927.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3928.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3929.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3930.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3931.bmp
no face found !!!
/home/harry/Downloads/TA/release/Data/Images/img_3932.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3933.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_3934.bmp
no face found !!!
/home/harry/Downlo

(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4033.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4034.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4035.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4036.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4037.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4038.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4039.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4040.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4041.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4042.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4043.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4044.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4045.bmp
(68, 2)
(2278,)
/home/harry/Downloads/

(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4144.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4145.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4146.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4147.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4148.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4149.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4150.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4151.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4152.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4153.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4154.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4155.bmp
(68, 2)
(2278,)
/home/harry/Downloads/TA/release/Data/Images/img_4156.bmp
(68, 2)
(2278,)
/home/harry/Downloads/

In [28]:
all_height

[1.7084282921856682,
 1.7480021672564132,
 1.6820583745973285,
 1.6901029486533798,
 1.7686861841269976,
 1.7190293455269763,
 1.7391054540056783,
 1.765441954567842,
 1.6992764927741242,
 1.6740763155120186,
 1.7250661357233301,
 1.7322559276985074,
 1.682828769382622,
 1.6725930409435552,
 1.719048093759739,
 1.7231412934242036,
 1.7370761024720633,
 1.706731186828832,
 1.7445630096007114,
 1.7203528100168435,
 1.7232934470733579,
 1.7258349294965012,
 1.756242821188092,
 1.726471945857654,
 1.7933395703508996,
 1.7780046024873268,
 1.7064495853540416,
 1.5559549886023727,
 1.7237405328823094,
 1.7084282921856682,
 1.6827918585454458,
 1.6356033505091385,
 1.737040266596837,
 1.7182719823527244,
 1.762067846405412,
 1.7313181541150537,
 1.734305730589917,
 1.7394898359077726,
 1.7270961647697756,
 1.7137918952498206,
 1.7258909421496722,
 1.7167406246949137,
 1.726893812675193,
 1.7110818839419037,
 1.7210237821374983,
 1.6958053962313102,
 1.7167777486226945,
 1.714729984640702,
 1.

In [29]:
all_weight

[90.69587736153329,
 91.77952658983644,
 95.56615414313279,
 88.1748270925496,
 125.35722273372863,
 87.78518606735739,
 107.96281247988092,
 92.63630720797671,
 105.97370314251246,
 86.04860000673357,
 99.09490666638007,
 103.97951202474954,
 89.59758883912158,
 87.0599114144063,
 91.48084268550019,
 95.93728421684668,
 101.11751829199419,
 84.98757666791258,
 97.48498788719961,
 94.30899774291619,
 96.77877455219108,
 88.77172028144226,
 130.83507964201564,
 100.96819068302034,
 136.4943954168809,
 93.07645370003613,
 107.40973501901905,
 66.50518517005786,
 86.5762681457252,
 90.69587736153329,
 115.28463582369851,
 61.153605488052726,
 109.46866670809594,
 79.05838255310677,
 125.55332459562898,
 95.56593370716334,
 107.90315298891278,
 110.31303989215546,
 99.31242668809882,
 95.93175208679565,
 90.43816193068753,
 80.90025988709463,
 90.60127215743105,
 86.01369483421634,
 105.37084012787963,
 95.80639418502196,
 99.30804058419173,
 96.60262906985743,
 98.50226682490629,
 85.4702

In [30]:
all_bmi

[31.072994283141103,
 30.04130630367381,
 33.80346730352789,
 30.90899486913779,
 40.08591361567702,
 29.677059023276065,
 35.70314784021185,
 29.721013276972158,
 36.68944279376289,
 30.694691796044186,
 33.29648447987962,
 34.65971015080666,
 31.633481993213508,
 31.061621035307347,
 30.958386771942816,
 32.309848888116385,
 33.530813762014,
 29.179796472900353,
 32.00599301508698,
 31.856060099165305,
 32.59642367940551,
 29.800762950725463,
 42.399918689932605,
 33.87769372992846,
 42.45806903044734,
 29.455831733624414,
 36.89737749254078,
 27.49069771184832,
 29.109752962079263,
 31.072994283141103,
 40.71942103524645,
 22.875872912209868,
 36.29684255206806,
 26.776491511240604,
 40.44645754408012,
 31.87820994229942,
 35.877984258199184,
 36.42593356756735,
 33.30695806189896,
 32.66033601159324,
 30.377549646310328,
 27.447928113825277,
 30.3945965703938,
 29.38577410594258,
 35.57368435238771,
 33.325170978055546,
 33.69460995665097,
 32.84175975560971,
 32.96077527848034,
 2

In [31]:
data_df['Predicted Weight'] = all_weight

In [32]:
data_df['Predicted Height'] = all_height

In [33]:
data_df['Predicted BMI'] = all_bmi

In [34]:
data_df

,name,path,Unnamed: 0,bmi,gender,weight,height,Predicted Weight,Predicted Height,Predicted BMI
0,img_3700.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,3700,48.277006,Male,144,1.73,90.695877,1.708428,31.072994
1,img_3701.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,3701,29.834105,Male,85,1.69,91.779527,1.748002,30.041306
2,img_3702.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,3702,36.019158,Female,97,1.64,95.566154,1.682058,33.803467
3,img_3703.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,3703,28.032301,Female,70,1.58,88.174827,1.690103,30.908995
4,img_3704.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,3704,48.779592,Male,158,1.80,125.357223,1.768686,40.085914
...,...,...,...,...,...,...,...,...,...,...
501,img_4201.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,4201,34.078947,Male,115,1.84,96.535323,1.725790,32.418907
502,img_4202.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,4202,34.564776,Female,94,1.65,83.677832,1.680118,29.681727
503,img_4203.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,4203,27.432362,Female,69,1.59,83.606182,1.660213,30.320717
504,img_4204.bmp,/home/harry/Downloads/TA/release/Data/Images/i...,4204,40.492800,Male,125,1.76,112.103734,1.770534,35.756300


In [35]:
df_weight = pd.DataFrame(data_df, columns = ['weight', 'Predicted Weight'])
df_weight

,weight,Predicted Weight
0,144,90.695877
1,85,91.779527
2,97,95.566154
3,70,88.174827
4,158,125.357223
...,...,...
501,115,96.535323
502,94,83.677832
503,69,83.606182
504,125,112.103734


In [36]:
df_height = pd.DataFrame(data_df, columns = ['height', 'Predicted Height'])
df_height

,height,Predicted Height
0,1.73,1.708428
1,1.69,1.748002
2,1.64,1.682058
3,1.58,1.690103
4,1.80,1.768686
...,...,...
501,1.84,1.725790
502,1.65,1.680118
503,1.59,1.660213
504,1.76,1.770534


In [37]:
df_bmi = pd.DataFrame(data_df, columns = ['bmi', 'Predicted BMI'])
df_bmi

,bmi,Predicted BMI
0,48.277006,31.072994
1,29.834105,30.041306
2,36.019158,33.803467
3,28.032301,30.908995
4,48.779592,40.085914
...,...,...
501,34.078947,32.418907
502,34.564776,29.681727
503,27.432362,30.320717
504,40.492800,35.756300


In [38]:
df_weight.to_csv('weight.csv', index=False)

In [39]:
df_height.to_csv('height.csv', index=False)

In [40]:
df_bmi.to_csv('bmi.csv', index=False)